In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# 1. Find Latest Session
log_dir = '../logs/balance/' # Adjust relative path if needed
all_sessions = sorted(glob.glob(os.path.join(log_dir, 'session_*')))
if not all_sessions:
    print("No logs found!")
else:
    latest_folder = all_sessions[-1]
    csv_file = os.path.join(latest_folder, 'telemetry.csv')
    print(f"Loading data from: {latest_folder}")

    # 2. Load Data
    try:
        df = pd.read_csv(csv_file)
        # Filter for ACTIVE state only (State 20 in your code)
        # This removes the setup/calibration noise
        df_active = df[df['state'] >= 20].copy() 
        
        if len(df_active) < 10:
            print("Warning: Not enough 'Active' balancing data found. showing all data.")
            df_active = df

        # Convert Time to Seconds starting at 0
        df_active['t'] = (df_active['timestamp_ms'] - df_active['timestamp_ms'].iloc[0]) / 1000.0

        # --- PLOT 1: Performance (Angle vs Time) ---
        plt.figure(figsize=(12, 6))
        plt.subplot(2, 1, 1)
        plt.plot(df_active['t'], df_active['pend_angle'], 'b', label='Pendulum Error')
        plt.axhline(0, color='k', linestyle='--', alpha=0.5)
        plt.title("Stability Check (Pendulum Angle)")
        plt.ylabel("Angle (deg)")
        plt.legend()
        plt.grid(True)

        # --- PLOT 2: Control Effort (Motor vs Time) ---
        plt.subplot(2, 1, 2)
        plt.plot(df_active['t'], df_active['motor_angle'], 'r', label='Motor Position')
        plt.axhline(45, color='r', linestyle=':', label='Limit')
        plt.axhline(-45, color='r', linestyle=':')
        plt.title("Control Effort (Motor Position)")
        plt.ylabel("Angle (deg)")
        plt.xlabel("Time (s)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # --- PLOT 3: Phase Portrait (The Tuning Tool) ---
        # This is the "Secret Weapon" for tuning D-Term.
        # X-axis: Angle (P), Y-axis: Velocity (D)
        # Calculate Velocity
        dt = np.mean(np.diff(df_active['t']))
        velocity = np.gradient(df_active['pend_angle'], dt)

        plt.figure(figsize=(8, 8))
        plt.plot(df_active['pend_angle'], velocity, 'g-', alpha=0.5)
        plt.plot(df_active['pend_angle'].iloc[0], velocity[0], 'ro', label='Start')
        plt.plot(df_active['pend_angle'].iloc[-1], velocity[-1], 'kx', label='End')
        plt.title("Phase Portrait (Angle vs Velocity)")
        plt.xlabel("Angle Error (deg)")
        plt.ylabel("Angular Velocity (deg/s)")
        plt.axvline(0, color='k', linestyle='--')
        plt.axhline(0, color='k', linestyle='--')
        plt.grid(True)
        plt.legend()
        
        print("INTERPRETATION:")
        print("- Spiraling OUT? Increase Kd (Damping)")
        print("- Spiraling IN? Good.")
        print("- Flat Oval? Too much Kd (Vibrating)")
        plt.show()

    except Exception as e:
        print(f"Error analyzing: {e}")

Loading data from: ../logs/balance/session_20260115_120502
Error analyzing: 'state'
